In [212]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

In [97]:
# Прочитаем CSV файл
ondoarding = pd.read_csv(r'C:\Users\kayum\OneDrive\Рабочий стол\simple_interview_events.csv')
ondoarding

,user_id,event_type,event_time,event_params
0,32001,onboarding_start,2024-01-01T00:01:40,"{""funnel_type"": ""female""}"
1,99564,onboarding_start,2024-01-01T00:01:53,"{""funnel_type"": ""male""}"
2,32001,profile_start,2024-01-01T00:01:58,{}
3,99564,profile_start,2024-01-01T00:02:07,{}
4,71575,onboarding_start,2024-01-01T00:02:18,"{""funnel_type"": ""female""}"
...,...,...,...,...
346323,54690,paywall_show,2024-04-01T00:01:40,{}
346324,11354,paywall_show,2024-04-01T00:02:51,{}
346325,64903,paywall_show,2024-04-01T00:04:27,{}
346326,41417,paywall_show,2024-04-01T00:05:01,{}


## 1. События онбординга

### Разбираемся с воронкой

In [100]:
# Создаем список, чтобы отсортировать потом воронку в нужном порядке
# Также список понадобится, чтобы взять из файла только события воронки
order = ['onboarding_start', 
         'profile_start',
         'email_submit', 
         'paywall_show', 
         'payment_done']

# Считаем количество уникальных пользователей по user_id на каждом этапе воронки
count = ondoarding.query("event_type in @order")\
          .groupby(['event_type']).agg({'user_id': 'nunique'}).reset_index()

# Применяем сортировку
count['event_type'] = pd.Categorical(count['event_type'], 
                                     categories=order, 
                                     ordered=True)
count = count.sort_values('event_type').reset_index(drop=True)

# Добавляем столбец с процентом перехода с каждого этапа воронки на следующий
count['conversion_rate_by_step'] = round(count['user_id'] / count['user_id']
                                         .shift(1) * 100, 2)

# Для первого этапа воронки устанавливаем 100%, это начальная точки отсчета
count.loc[0, 'conversion_rate_by_step'] = 100


'''
Теперь добавим сквозную воронку. 
То есть процент перехода пользователей на этапы от самого первого, 
а не просто предыдущего.
'''
# Начальное количество пользователей
onboarding_start_count = (ondoarding.query("event_type == ['onboarding_start']")\
                                   .groupby(['event_type'])
                                   .agg({'user_id': 'nunique'})
                                   .reset_index().iloc[0, 1]) 

count['conversion_rate_from_the_beginning'] = (count['user_id'] /
                                              onboarding_start_count * 100)

'''
Мы посмотрели воронку по нашим пользователям за весь период. С первого этапа 
до последнего доходит 5,43% пользователей - сквозная воронка. 
В таблице также видна конверсия каждого из этапов. 

В столбце conversion_rate_by_step считается переход с каждого шага на следующий,
то есть конверсия определенного этапа с его предыдущего. 
По нему мы видим, что шаги email_submit (65227/83051 = 78.54%)
и payment_done имеют наименьшую конверсию. Возможно, это является слабым местом
воронки, которое можно было бы доработать, но нужно дополнительно разбираться.
Потому что 78.54% и 8.05% может быть и не являются низкой конверсией, 
а вполне в пределах нормы.
'''
count

,event_type,user_id,conversion_rate_by_step,conversion_rate_from_the_beginning
0,onboarding_start,100000,100.00,100.000
1,profile_start,83051,83.05,83.051
2,email_submit,65227,78.54,65.227
3,paywall_show,67478,103.45,67.478
4,payment_done,5430,8.05,5.430


In [111]:
'''
Меня смутило, что с шага email_submit на paywall_show пользователей стало больше.
По логике воронки она должна сужаться к низу. Есть вероятность, что мы 
не зафиксировали все события email_submit. Или может логика воронки подразумевает
показ paywall не всегда с почтой, тут тоже дополнительно нужно было бы разобраться.
'''

# Преобразуем столбец event_time в формат даты и времени
ondoarding['event_time'] = pd.to_datetime(ondoarding['event_time'])

# Извлекаем месяц
ondoarding['event_month'] = ondoarding['event_time'].dt.month

# Выполним группировку по месяцам и типам событий
monthly_info = ondoarding.query("event_type in @order")\
          .groupby(['event_month', 'event_type'])\
          .agg({'user_id': 'nunique'}).reset_index()

# Применяем сортировку
monthly_info['event_type'] = pd.Categorical(monthly_info['event_type'], 
                                     categories=order, 
                                     ordered=True)
monthly_info = monthly_info.sort_values(['event_month', 'event_type'])\
                           .reset_index(drop=True)
    
# Рассчитаем процент перехода (conversion_rate_by_step) в рамках каждого месяца
monthly_info['conversion_rate_by_step'] = monthly_info.groupby('event_month', 
                                                               group_keys=False)['user_id']\
            .apply(lambda x: round(x / x.shift(1) * 100, 2))

# Для первой строки каждого месяца установим значение 100%, так как это начальный этап
monthly_info['conversion_rate_by_step'].fillna(100, inplace=True)

'''
Попробовал посмотреть в каждом ли месяце показов пэйвола больше, чем email_submit.
Оказалось что да. Возможно это действительно норма для этой воронки и сервис
позволяет как то без почты показывать пейвол. В любом случае нужно было бы
выяснить что к чему:)
'''
monthly_info

,event_month,event_type,user_id,conversion_rate_by_step
0,1,onboarding_start,34121,100.00
1,1,profile_start,28276,82.87
2,1,email_submit,22262,78.73
3,1,paywall_show,23051,103.54
4,1,payment_done,1836,7.96
5,2,onboarding_start,31490,100.00
6,2,profile_start,26137,83.00
7,2,email_submit,20431,78.17
8,2,paywall_show,21203,103.78
9,2,payment_done,1765,8.32


### Выбираем 3 наиболее перспективных теста

In [124]:
# Посчитаем количество пользователей в каждой из тестовых групп
ondoarding.query("event_type == 'experiment_exposure'")\
          .groupby('event_params').agg({'user_id': 'nunique'})\
          .reset_index()

,event_params,user_id
0,"{""experiment_name"": ""exp_0"", ""experiment_group"": ""control""}",1514
1,"{""experiment_name"": ""exp_0"", ""experiment_group"": ""test""}",1445
2,"{""experiment_name"": ""exp_1"", ""experiment_group"": ""control""}",2074
3,"{""experiment_name"": ""exp_1"", ""experiment_group"": ""test""}",2099
4,"{""experiment_name"": ""exp_2"", ""experiment_group"": ""control""}",2866
5,"{""experiment_name"": ""exp_2"", ""experiment_group"": ""test""}",2865
6,"{""experiment_name"": ""exp_5"", ""experiment_group"": ""control""}",666
7,"{""experiment_name"": ""exp_5"", ""experiment_group"": ""test""}",680
8,"{""experiment_name"": ""exp_6"", ""experiment_group"": ""control""}",847
9,"{""experiment_name"": ""exp_6"", ""experiment_group"": ""test""}",843


In [89]:
count

,event_type,user_id,conversion_rate_by_step,conversion_rate_from_the_beginning
0,onboarding_start,100000,100.00,100.000
1,profile_start,83051,83.05,83.051
2,email_submit,65227,78.54,65.227
3,paywall_show,67478,103.45,67.478
4,payment_done,5430,8.05,5.430


У нас есть 9 проведенных тестов, но нет информации что именно тестировалось. 
Предположим, вводилось 9 каких то изменений, которые были направлены на улучшение воронки. С самого начала до самого конца.
То есть с onboarding_start до payment_done.

У нас сквозная конверсия воронки составляет 5.43%. 
Базово хочется отметить, что мы бы могли назвать самыми перспективными тесты, в которых было больше всего участников.
Потому что в таком случае мы сможем значимо уловить MDE меньшего процента.
Если мы задаем МДЕ 2%, то в каждой нашей группе должно быть примерно 2115 пользователей (при мощности 80% и значимости 5%), чтобы достоверно провести тест. 
У нас как раз 3 теста (1,2,8), количество пользователей в группах которых примерно равно или выше этого значения.
Однако в задаче сложно сказать, что 2% МДЕ это то, что нам нужно, в реальности хотелось бы изучить больше исторических данных и посмотреть волатильность метрики. 
Возможно, мы бы могли задать МДЕ 5% и тогда нам бы бы требовалось всего 360 пользователей на каждую группу. Поэтому нужно рассмотреть еще варианты определения 
перспективных тестов. 

Попробуем посчитать результаты АБ теста если бы он был направлен на улучшение сквозной воронки.




In [237]:
# Создаю отдельный df с экспериментами
experiment_exposure = ondoarding.query("event_type == 'experiment_exposure'")

# Оставлю в датасете только тех, кто был в какой либо группе любого эксперимента
tests_funnel = ondoarding.query("event_type == ['onboarding_start', 'payment_done']")\
                          .merge(experiment_exposure[['user_id', 'event_params']], 
                                 how = 'left', 
                                 on = 'user_id')\
                          .query("~event_params_y.isna()")\
                          .rename(columns = {'event_params_x':'event_params',
                                             'event_params_y':'tests'})

# Создаю датасет с информацией о том конвертировался пользователь или нет
# Также предварительно готовлю данные для расчета результата экспериментов
df = tests_funnel.query("event_type == 'onboarding_start'")
df = df.merge(tests_funnel.query("event_type == 'payment_done'")[['user_id', 'tests']], 
              how = 'left', 
              on = 'user_id')\
       .rename(columns = {'tests_x':'tests', 'tests_y':'converted'})
df['converted'] = df['converted'].notna()
df['experiment_group'] = df['tests'].str.extract(r'"experiment_group":\s*"([^"]+)"')
df['experiment_num'] = df['tests'].str.extract(r'"experiment_name":\s*"([^"]+)"')
df = df[['user_id', 'experiment_num', 'converted', 'experiment_group']]

def run_z_test_for_experiment(df, experiment_num):
    # Фильтрация данных по номеру эксперимента
    filtered_df = df.query(f"experiment_num == '{experiment_num}'")

    # Определение контрольной и тестовой групп
    control_group = filtered_df[filtered_df['experiment_group'] == 'control']['converted']
    test_group = filtered_df[filtered_df['experiment_group'] == 'test']['converted']

    # Подсчет успехов и размеров выборок
    success_a = control_group.sum()  # Конверсии в контрольной группе
    n_a = control_group.count()       # Размер выборки в контрольной группе

    success_b = test_group.sum()      # Конверсии в тестовой группе
    n_b = test_group.count()          # Размер выборки в тестовой группе

    # Выполнение Z-теста
    count = np.array([success_a, success_b])
    nobs = np.array([n_a, n_b])

    z_stat, p_value = proportions_ztest(count, nobs)

    return z_stat, p_value

# Вызов функции для расчета в цикле для всех экспериментов
experiment_nums = ['exp_0', 'exp_1', 'exp_2', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9']

for experiment_num in experiment_nums:
    z_statistic, p_value = run_z_test_for_experiment(df, experiment_num)
    
    print(f'Z-statistic for {experiment_num}: {z_statistic:.4f}')
    print(f'P-value for {experiment_num}: {p_value:.4f}')

Z-statistic for exp_0: 1.1436
P-value for exp_0: 0.2528
Z-statistic for exp_1: 0.2883
P-value for exp_1: 0.7731
Z-statistic for exp_2: -4.7788
P-value for exp_2: 0.0000
Z-statistic for exp_5: -0.1939
P-value for exp_5: 0.8462
Z-statistic for exp_6: -0.8525
P-value for exp_6: 0.3940
Z-statistic for exp_7: 0.0023
P-value for exp_7: 0.9982
Z-statistic for exp_8: 1.1704
P-value for exp_8: 0.2419
Z-statistic for exp_9: -0.5795
P-value for exp_9: 0.5623


In [257]:
# Показатели сквозных конверсий в тестах:
for experiment_num in experiment_nums:
    # Рассчитываем конверсию для тестовой группы
    conversion_rate = df.query(f"experiment_num == '{experiment_num}' & experiment_group == 'test'")['converted'].sum() / \
                      df.query(f"experiment_num == '{experiment_num}' & experiment_group == 'test'")['converted'].count()*100
    
    # Выводим результат
    print(f"Сквозная конверсия тестовых групп в эксперименте '{experiment_num}': {conversion_rate:.4f}")

    # Показатели сквозных конверсий в контроле:
for experiment_num in experiment_nums:
    # Рассчитываем конверсию для тестовой группы
    conversion_rate = df.query(f"experiment_num == '{experiment_num}' & experiment_group == 'control'")['converted'].sum() / \
                      df.query(f"experiment_num == '{experiment_num}' & experiment_group == 'control'")['converted'].count()*100
    
    # Выводим результат
    print(f"Сквозная конверсия контрольных групп в эксперименте '{experiment_num}': {conversion_rate:.4f}")


Сквозная конверсия тестовых групп в эксперименте 'exp_0': 5.5363
Сквозная конверсия тестовых групп в эксперименте 'exp_1': 6.8604
Сквозная конверсия тестовых групп в эксперименте 'exp_2': 10.9250
Сквозная конверсия тестовых групп в эксперименте 'exp_5': 7.9412
Сквозная конверсия тестовых групп в эксперименте 'exp_6': 8.1851
Сквозная конверсия тестовых групп в эксперименте 'exp_7': 8.0940
Сквозная конверсия тестовых групп в эксперименте 'exp_8': 5.3548
Сквозная конверсия тестовых групп в эксперименте 'exp_9': 5.5699
Сквозная конверсия контрольных групп в эксперименте 'exp_0': 6.5390
Сквозная конверсия контрольных групп в эксперименте 'exp_1': 7.0878
Сквозная конверсия контрольных групп в эксперименте 'exp_2': 7.2924
Сквозная конверсия контрольных групп в эксперименте 'exp_5': 7.6577
Сквозная конверсия контрольных групп в эксперименте 'exp_6': 7.0838
Сквозная конверсия контрольных групп в эксперименте 'exp_7': 8.0972
Сквозная конверсия контрольных групп в эксперименте 'exp_8': 6.1599
Скв

У меня возникли трудности с опеределением перспективного теста.
Как уже писал выше, начал бы отталкиваться от количества пользователей в выборке.
А потом углубился бы в сами результаты теста. Но значимых результатов не увидел ни в одном из проведенных тестов кроме второго.
В результатах второго теста действительно конверсия сильно выше в тестовой группе, чем в контрольной. 
Поэтому можно выделить перспективность второго теста и подумать о раскатке фичи.

Чтобы определить перспективность тестов, хотелось бы понимать на какую метрику они направлены и что именно мы тестировали 
запуская их. 


## 2. Ежедневные задания

### Основная задача Simple — помочь пользователю в его пути к здоровому образу жизни и лучшей версии себя. Simple использует информацию из множества разных источников в продукте (ответы с онбординга, взаимодействие с фичами внутри приложения, данные с внешних устройств и т.д.) о конкретном пользователе, чтобы сформировать для него список заданий на ежедневной основе. Пользователь может согласиться с заданиями или попросить поменять их, но поменять их можно только один раз в день. На следующий день у пользователя появляются новые задания.  


### Как бы вы оценили то, насколько хорошо работает выдача этих ежедневных заданий? Ответ может быть представлен в любом формате в зависимости от выбранного подхода, но должен быть обоснован. 

Оценка работы системы выдачи ежедневных заданий в приложении:

1. Анализ пользовательской вовлеченности:

    - Метрика выполнения заданий: 
    Один из самых очевидных индикаторов успешности системы заданий — 
    это процент выполненных заданий пользователями. Если большинство пользователей выполняет задания, 
    это говорит нам о том, что они считают их полезными и достижимыми. 
    Однако, низкий процент выполнения может говорить о том, что задания слишком сложны, 
    не соответствуют мотивации или целям пользователя, или недостаточно персонализированы.
    Можно смотреть как выполнения отдельных упражнений в рамках кадого дня, если даётся несколько.
    И также смотреть выполнение в целом по дням.
    Это поможет обнаружить и сезонность при его наличии. То есть в понедельник, например, 
    выполняют 10 заданий из 10. А в середине недели число падает до 5 из 10. И снова отрастает к выходным.
    Бизнесу будет полезно знать о наличии таких циклов, потому что в будущем можно будет разрабатывать
    новые задания с учетом этого фактора.
    
    - Частота смены заданий: 
    Если пользователи часто запрашивают изменения заданий, это может указывать 
    на недостаточную персонализацию или неподходящие задачи и подтолкнёт нас на проведение дополнительных
    исследований.
    В качестве этих доп. исследований важным будет анализировать причины, по которым задания меняются, 
    выделил это как 4ый пункт отдельно.
    
2. Качество персонализации выдаваемых заданий:

    - Опросы и отзывы пользователей: 
    Прямые опросы или отзывы после выполнения заданий могут дать важную информацию о том, 
    насколько пользователи считают задания релевантными и мотивирующими. 
    Вопросы могут включать оценку полезности задания, его достижимости и соответствия текущим целям пользователя.
    
    - Точность данных: 
    Важно оценить, насколько точно приложение использует внешние данные (с онбординга, устройств, истории поведения) 
    для формирования заданий. 
    Если задания базируются на устаревших или неверных данных, это может снижать их полезность.
    Здесь имеется в виду просто то, что данные о пользователях лучше всего собирать регулярно.
    Условно было написано, что пользователь весит 80кг, а он хотел был 75.
    Потом он достиг цели, сменил свой весь на 75 и поставил новую цель 70.
    А мы пришлём ему задания, чтобы добиться цели 75, будет нехорошо.
    
3. Долгосрочные метрики успеха:

    - Изменения в привычках и здоровье пользователя: 
    "Основной целью Simple является помощь пользователю в его пути к здоровью и лучшей версии себя."
    Поэтому важными метриками являются изменения в физическом и эмоциональном состоянии пользователей на протяжении времени. 
    Можно отслеживать такие индикаторы, как стабильность выполнения заданий, 
    улучшение параметров здоровья (вес, уровень активности и т.д.).

4. Анализ отказов:

    Причины отказа от заданий: Нужно провести анализ тех случаев, когда пользователи не только просят изменить задания, 
    но и отказываются их выполнять. 
    Какие типы задач чаще всего вызывают отказ? 
    Это могут быть задачи, которые воспринимаются как слишком трудные, слишком простые или не соответствующие ожиданиям.
    Опросы также могут помочь выявить эти причины и в дальнейшем проводить работу с ними.

5. Тестирование A/B:

    Эксперименты с различными подходами к выдаче заданий: 
    Проведение A/B тестов с разными типами персонализированных заданий поможет понять, 
    какой набор рекомендаций или задач лучше всего работает для разных типов пользователей. 
    Например, можно протестировать более жёсткие задания против более мягких, или задания, 
    сфокусированные на разных аспектах здоровья (например, диета против физической активности).
    
Итог:
    Оценка эффективности выдачи ежедневных заданий должна основываться на комбинации количественных (выполнение заданий, частота смены заданий) 
    и качественных данных (отзывы пользователей, долгосрочные улучшения здоровья). 
    Такой подход позволит понять, насколько хорошо система выполняет свою основную цель — 
    помочь пользователю улучшить свой образ жизни и достичь поставленных целей.
    И важно понимать, что все эти метрики будут хорошо работать вместе. 
    Они дополняют друг друга и в совокупности несут
    полезную информацию для потенциальных улучшений продукта.

### Какие улучшения и изменения вы могли бы предложить менеджеру в зависимости от результатов, полученных в п. 1? Объясните, почему предлагаете именно такие идеи. 

Предложения по улучшению системы выдачи ежедневных заданий могут сильно зависеть от результатов, полученных в первом пункте. Рассмотрим возможные сценарии и подходы, которые можно предложить менеджеру:

1. Низкий процент выполнения заданий

Проблема: Пользователи редко выполняют задания, что может свидетельствовать о том, что задания кажутся слишком сложными, неподходящими или слишком общими.

Решения:

- Улучшение персонализации: Если задания недостаточно индивидуализированы, можно рассмотреть улучшение алгоритмов персонализации на основе данных о пользователе. Например, задействовать другие (возможно, более сложные) алгоритмы для учёта большего количества факторов (состояние здоровья, уровень активности, цели пользователя). Это поможет нам выдавать более релевантные и персонализированные задания, которые будут лучше соответствовать мотивации пользователя.
- Интервальная сложность: Предоставление пользователю выбора сложности задач — например, лёгкий, средний или сложный уровень — позволит ему лучше адаптировать задания к своему текущему состоянию или уровню энергии.
- Микрозадачи: Если задания слишком большие или кажутся трудновыполнимыми, можно предложить пользователям микрозадачи — меньшие шаги, которые проще выполнить. Это может помочь в создании привычки и постепенном увеличении сложности заданий.
Почему: Если пользователи видят задания как слишком трудные или нерелевантные, это снижает их мотивацию. Улучшение персонализации и адаптивности позволит повысить интерес и мотивацию к выполнению заданий.

2. Частые запросы на изменение заданий

Проблема: Пользователи часто просят изменить задания, что может говорить о недостаточной релевантности или точности алгоритмов формирования задач.

Решения:

- Анализ запросов на изменения: Провести более глубокий анализ того, какие именно задания чаще всего вызывают запросы на изменения. Возможно, некоторые типы задач (например, диетические рекомендации или задания по физической активности) требуют более продуманной адаптации под конкретного юзера. Можно улучшить систему так, чтобы она лучше понимала предпочтения пользователя, что снизит количество запросов на изменения.
- Динамическая адаптация: Добавить возможность динамической корректировки заданий в течение дня в зависимости от поведения пользователя. Например, если пользователь не смог выполнить утреннюю тренировку, система могла бы предложить ему более лёгкое задание вечером, что повысит вероятность выполнения хотя бы части плана.
- Частичная замена заданий: Вместо полной замены всех заданий пользователю можно предложить возможность частично менять задания. Например, если одно из трёх заданий не подходит, можно заменить только его, оставив другие два.
Почему: Постоянные запросы на изменения указывают на необходимость улучшения точности выдачи заданий пользователям. Адаптация на основе поведения и предпочтений пользователя поможет повысить его вовлечённость.

3. Позитивные отзывы, но низкий долгосрочный прогресс

Проблема: Пользователи хорошо оценивают задания и выполняют их, но долгосрочные изменения в привычках и здоровье остаются незначительными.

Решения:

- Долгосрочные цели и прогресс: Включить в систему функции отслеживания долгосрочных целей. Например, показывать пользователю его прогресс за более продолжительные периоды (недели, месяцы), связывая его текущие достижения с долгосрочными целями.
- Интеграция коучинга: Предложить пользователю консультации с виртуальными коучами или экспертами, которые помогут корректировать его план и мотивировать его на достижение более сложных целей.
- Обратная связь и рефлексия: Включить систему регулярной рефлексии. Например, каждую неделю предлагать пользователю оценивать свой прогресс, что поможет усилить мотивацию и осознание изменений в его поведении.
Почему: Если пользователи не видят долгосрочного прогресса, это может подорвать их мотивацию. Интеграция долгосрочных целей и функций отслеживания прогресса поможет пользователям оставаться на пути к здоровому образу жизни.

4. Высокий процент выполнения, но низкая активность пользователей (невысокое взаимодействие)

Проблема: Пользователи выполняют задания, но не проявляют высокой активности в приложении (редко открывают приложение, не взаимодействуют с другими функциями).

Решения:

- Геймификация: Добавить элементы геймификации, такие как система наград, достижения, уровни. Это может мотивировать пользователей чаще открывать приложение и взаимодействовать с различными его функциями.
- Социальное взаимодействие: Внедрить функции социального взаимодействия, такие как возможность делиться успехами с друзьями, участвовать в групповых челленджах или поддерживать других пользователей.
- Контент и обучение: Включить обучающий контент по здоровому образу жизни (статьи, видео, подкасты), который будет предлагаться пользователям в качестве дополнительных ежедневных задач или мотивации.
Почему: Для удержания пользователей важно не только выполнение заданий, но и создание вовлекающего пользовательского опыта. Геймификация и социальное взаимодействие могут усилить активность и мотивацию пользователей.

5. Высокие показатели выполнения и отзывов, стабильный прогресс

Проблема: Если система показывает отличные результаты по всем ключевым метрикам (высокие проценты выполнения, позитивные отзывы, долгосрочный прогресс), можно сфокусироваться на масштабировании.

Решения:

- Масштабирование функционала: Добавить новые типы заданий, которые помогут пользователям разнообразить их путь к здоровью. Например, задания, связанные с эмоциональным состоянием (медитации, работа с сознанием и т.д).
- Расширение интеграций: Внедрить интеграции с новыми устройствами и источниками данных (например, умные часы, трекеры сна и т.д.), чтобы задания становились ещё более точными и разнообразными.
- Аналитика и рекомендации: Улучшить систему рекомендаций, предлагая пользователям более сложные задачи по мере того, как они прогрессируют.
Почему: В этом случае основной акцент должен быть на поддержании успеха и дальнейшем расширении возможностей приложения для улучшения пользовательского опыта.

Общий вывод:
Предлагаемые улучшения направлены на повышение персонализации, вовлеченности и долгосрочной мотивации пользователей. В зависимости от результатов, предложенные шаги могут помочь решить выявленные проблемы и увеличить успех приложения в достижении его главной цели — помогать пользователям стать более здоровыми и улучшить свой образ жизни.

### Представьте, что менеджер только приходит к вам с идеей реализации таких ежедневных заданий — еще даже нет макетов, и тем более разработки. Как бы вы подошли к оценке перспективности такой фичи?

Оценка перспективности идеи ежедневных заданий требует анализа с разных сторон, включая потребности пользователей, рыночную ситуацию и потенциальные сложности внедрения. Подход к такой оценке может включать несколько этапов:

1. Исследование потребностей и мотивации пользователей
Что сделать:

Провести исследование целевой аудитории (опросы, интервью, фокус-группы), чтобы понять, какие цели и проблемы пользователи связывают с улучшением своего здоровья и как они воспринимают ежедневные задачи.
Изучить, насколько часто пользователи хотели бы получать задания, как они относятся к необходимости следовать определённым рекомендациям, и насколько они готовы выполнять задачи, предлагаемые приложением.
Почему это важно: Без понимания пользовательской мотивации и боли идея может не получить нужного отклика. Например, для одних пользователей ежедневные задания могут быть источником мотивации, а для других — излишним давлением. Оценка желаемой частоты и объёма задач поможет скорректировать концепцию. Также мы сможем существующих пользователей нашего прилоежения поделить на разные группы. Затем, если решим выкатывать такой функционал, можно будет работать с по разному с каждой группой пользователей:
- потенциальные любители ежедневных заданий;
- те, кому ни в коем случае нельзя давать ежедневные задания и тд.


2. Анализ конкурентов и рынка.

Что сделать:

Проанализировать существующие решения на рынке приложений для здоровья и похудения, таких как MyFitnessPal, Noom, Fitbit и другие, чтобы понять, как они реализуют функционал, связанный с заданиями или рекомендациями.
Изучить рыночные тренды, связанные с персонализацией здоровья (например, популярность рекомендаций на основе данных с устройств, тенденция к более гибким и адаптивным программам).
Почему это важно: Изучение конкурентов позволит определить, есть ли уже аналогичные функции, и насколько они успешны. Это также поможет выделить возможности для улучшения или внедрения уникальных функций, которые могут повысить конкурентоспособность продукта.

3. Анализ пользы для пользователей и привнесенной ценности.

Что сделать:

Оценить, какую пользу ежедневные задания могут приносить пользователю. Например, будут ли они направлены на улучшение физического здоровья (активность, питание) или ментального (осознанность, управление стрессом)?
Оценить долгосрочную ценность: как ежедневные задания будут влиять на привычки пользователей, помогут ли они достичь долгосрочных целей, и как приложение сможет поддерживать прогресс.
Почему это важно: Важно, чтобы ежедневные задания имели не только краткосрочный, но и долгосрочный эффект для пользователей, укрепляя их приверженность здоровому образу жизни. Пользователи, которые видят реальную пользу от задания, будут с большей вероятностью продолжать использовать приложение.

4. Оценка сложности разработки и реализации.

Что сделать:

Оценить технические сложности разработки механизма персонализации заданий, который будет учитывать данные пользователя из разных источников (например, данные с фитнес-трекеров, ответы на опросы, поведение внутри приложения).
Рассмотреть возможности использования алгоритмов машинного обучения или рекомендационных систем для более точной персонализации. Оценить затраты и время на разработку таких решений.
Почему это важно: Персонализация заданий требует интеграции данных и настройки сложных алгоритмов. Чем точнее будут задания, тем выше будет их эффективность. Оценка технических трудностей на раннем этапе позволит рассчитать временные и финансовые затраты на внедрение.

5. Гипотезы и тестирование идеи.

Что сделать:

Построить несколько гипотез о том, как ежедневные задания могут влиять на ключевые показатели продукта (например, улучшение удержания пользователей, увеличение времени взаимодействия с приложением, повышение уровня выполнения целей).
Рассмотреть возможность быстрого MVP (minimum viable product): начать с простого решения, например, с шаблонных ежедневных заданий или рекомендаций, и протестировать их на небольшой группе пользователей.
Запустить A/B тестирование, чтобы проверить, как наличие ежедневных заданий влияет на поведение пользователей по сравнению с контролем (отсутствие заданий).
Почему это важно: MVP и тестирование позволят быстро и с минимальными затратами проверить жизнеспособность идеи, понять, как пользователи реагируют на задания, и выявить проблемные места.

6. Оценка рисков.

Что сделать:

Рассмотреть возможные риски: например, перегрузка пользователей задачами, недостаточная персонализация, потеря интереса к заданиям через несколько недель.
Оценить правовые аспекты использования данных пользователей, особенно в контексте здоровья (Законы о защите данных, например).
Почему это важно: Предсказание и управление рисками на раннем этапе поможет избежать проблем в будущем и сохранить лояльность пользователей.

7. Финансовая и бизнес-оценка
Что сделать:

Оценить, каким образом эта функция будет влиять на монетизацию продукта (например, повышение удержания пользователей, увеличение вовлечённости, возможность добавления премиум-версий с расширенными заданиями).
Рассчитать возможные затраты на разработку и поддержку функции, а также прогнозируемую окупаемость.
Почему это важно: Важно понять, как ежедневные задания повлияют на ключевые бизнес-показатели (например, ARPU — средняя прибыль с одного пользователя, LTV - пожизненная ценность клиента) и стоит ли инвестировать в разработку этой функции.

Итог:
Этапы оценки перспективности идеи ежедневных заданий включают:

- Исследование пользовательских потребностей,
- Анализ конкурентов,
- Оценку долгосрочной пользы для пользователя,
- Технические сложности,
- Тестирование гипотез через MVP,
- Оценку рисков и влияние на бизнес.

Такая оценка поможет менеджеру понять, насколько востребована функция, какие проблемы могут возникнуть, и как можно оптимизировать её разработку и внедрение.